# Python 3 : Le BN pour le traitement du son

## Le son au format audio WAV
L'écriture et la lecture du contenu d'un fichier audio au format WAV est simplifiée par l'utilisation du module wave.

### Ecriture

In [1]:
# script audio.py
# (C) Fabrice Sincère ; Jean-Claude Meilland
import wave
import math
import binascii

print("Création d'un fichier audio au format WAV (PCM 8 bits stéréo 44100 Hz)")
print("Son de forme sinusoïdale sur chaque canal\n")

NomFichier = 'son.wav'
Monson = wave.open(NomFichier,'w') # instanciation de l'objet Monson

nbCanal = 2    # stéreo
nbOctet = 1    # taille d'un échantillon : 1 octet = 8 bits
fech = 44100   # fréquence d'échantillonnage

frequenceG = float(input('Fréquence du son du canal de gauche (Hz) ? '))
frequenceD = float(input('Fréquence du son du canal de droite (Hz) ? '))
niveauG = float(input('Niveau du son du canal de gauche (0 à 1) ? '))
niveauD = float(input('Niveau du son du canal de droite (0 à 1) ? '))
duree = float(input('Durée (en secondes) ? '))

nbEchantillon = int(duree*fech)
print("Nombre d'échantillons :",nbEchantillon)

parametres = (nbCanal,nbOctet,fech,nbEchantillon,'NONE','not compressed')# tuple
Monson.setparams(parametres)    # création de l'en-tête (44 octets)

# niveau max dans l'onde positive : +1 -> 255 (0xFF)
# niveau max dans l'onde négative : -1 ->   0 (0x00)
# niveau sonore nul :                0 -> 127.5 (0x80 en valeur arrondi)

amplitudeG = 127.5*niveauG
amplitudeD = 127.5*niveauD

print('Veuillez patienter...')
for i in range(0,nbEchantillon):
    # canal gauche
    # 127.5 + 0.5 pour arrondir à l'entier le plus proche
    valG = wave.struct.pack('B',int(128.0 + amplitudeG*math.sin(2.0*math.pi*frequenceG*i/fech)))
    # canal droit
    valD = wave.struct.pack('B',int(128.0 + amplitudeD*math.sin(2.0*math.pi*frequenceD*i/fech)))
    Monson.writeframes(valG + valD) # écriture frame

Monson.close()

Fichier = open(NomFichier,'rb')
data = Fichier.read()
tailleFichier = len(data)
print('\nTaille du fichier',NomFichier, ':', tailleFichier,'octets')
print("Lecture du contenu de l'en-tête (44 octets) :")
print(binascii.hexlify(data[0:44]))
print("Nombre d'octets de données :",tailleFichier - 44)
Fichier.close()

Création d'un fichier audio au format WAV (PCM 8 bits stéréo 44100 Hz)
Son de forme sinusoïdale sur chaque canal

Fréquence du son du canal de gauche (Hz) ? 440
Fréquence du son du canal de droite (Hz) ? 880
Niveau du son du canal de gauche (0 à 1) ? 0.5
Niveau du son du canal de droite (0 à 1) ? 0.5
Durée (en secondes) ? 5
Nombre d'échantillons : 220500
Veuillez patienter...

Taille du fichier son.wav : 441044 octets
Lecture du contenu de l'en-tête (44 octets) :
b'52494646ccba060057415645666d7420100000000100020044ac0000885801000200080064617461a8ba0600'
Nombre d'octets de données : 441000


Le script crée le fichier son.wav dans le répertoire courant (mais on peut aussi choisir un emplacement en spécifiant le chemin relatif à partir de ce bloc-note : Mon dossier/son.wav').

### Ecouter le son
Vous pouvez écouter le son ainsi généré en double-cliquant sur le fichier son.wav dans l'environnement jupyter ou en local avec un lecteur multimédia quelconque (VLC par exemple).
Notez que Python sait faire directement une lecture audio avec le module adéquat (module externe pygame.mixer, module ossaudiodev sous Linux, module winsound sous Windows, etc...)

Le module pygame est un module externe de création de jeux vidéo en 2D.
pygame contient un sous module pygame.mixer qui permet de charger et de lire des musiques ou des sons dans plusieurs formats (mp3, ogg, wav...).

In [ ]:
import pygame
pygame.mixer.init()
pygame.mixer.Sound("son.wav").play()
while pygame.mixer.get_busy():
    # lecture en cours
    pass

### Editeur audio
On peut compléter l'étude du fichier son.wav avec un éditeur de sons tel que Audacity.

### Editeur hexadécimal
Un éditeur hexadécimal est aussi utile : https://www.onlinehexeditor.com/ OU https://hexed.it/

> A l’aide des documents ressources <a href="pdf/FormatFichiersAudio.pdf">Format de fichiers audio</a> et <a href="pdf/FormatWAV.pdf">Format WAV</a> «», analyser le fichier son produit et le code Python qui l’a généré…

### Lecture
Le script suivant fonctionne avec tous les fichiers audio au format WAV (PCM sans compression).

In [ ]:
# (C) Fabrice Sincère
import wave
import binascii

NomFichier = input('Entrer le nom du fichier : ')
Monson = wave.open(NomFichier,'r')	# instanciation de l'objet Monson

print("\nNombre de canaux :",Monson.getnchannels())
print("Taille d'un échantillon (en octets):",Monson.getsampwidth())
print("Fréquence d'échantillonnage (en Hz):",Monson.getframerate())
print("Nombre d'échantillons :",Monson.getnframes())
print("Type de compression :",Monson.getcompname())

TailleData = Monson.getnchannels()*Monson.getsampwidth()*Monson.getnframes()

print("Taille du fichier (en octets) :",TailleData + 44)
print("Nombre d'octets de données :",TailleData)

print("\nAffichage d'une plage de données (dans l'intervalle 0 -",Monson.getnframes()-1,")")

echDebut = int(input('N° échantillon (début) : '))
echFin = int(input('N° échantillon (fin) : '))

print("\nN° échantillon	Contenu")

Monson.setpos(echDebut)
plage = echFin - echDebut + 1
for i in range(0,plage):
    print(Monson.tell(),'\t\t',binascii.hexlify(Monson.readframes(1)))

Monson.close()

### Source :
http://fsincere.free.fr/isn/python/cours_python_ch9.php

## python-sonic - Programming Music with Python, Sonic Pi or Supercollider

Python-Sonic is a simple Python interface for Sonic Pi, which is a real great music software created by Sam Aaron (http://sonic-pi.net). 

At the moment Python-Sonic works with Sonic Pi. It is planned, that it will work with Supercollider, too.

If you like it, use it. If you have some suggestions, tell me (gkvoelkl@nelson-games.de).

## Installation

* First you need Python 3 (https://www.python.org, ) - Python 3.5 should work, because it's the development environment
* Then Sonic Pi (https://sonic-pi.net) - That makes the sound
* Modul python-osc (https://pypi.python.org/pypi/python-osc) - Connection between Python and Sonic Pi Server
* And this modul python-sonic - simply copy the source

Or try

In [ ]:
$ pip install python-sonic

That should work.

## Limitations

* You have to start _Sonic Pi_ first before you can use it with python-sonic
* Only the notes from C2 to C6

## Changelog

|Version       |                                                                                          |
|--------------|------------------------------------------------------------------------------------------|
|  0.2.0       | Some changes for Sonic Pi 2.11. Simpler multi-threading with decorator *@in_thread*. Messaging with *cue* and *sync*. |

## Examples

Many of the examples are inspired from the help menu in *Sonic Pi*.

In [ ]:
from psonic import *

The first sound

In [ ]:
play(70) #play MIDI note 70

Some more notes

In [ ]:
play(72)
sleep(1)
play(75)
sleep(1)
play(79) 

In more tratitional music notation

In [ ]:
play(C5)
sleep(0.5)
play(D5)
sleep(0.5)
play(G5) 

Play sharp notes like *F#* or dimished ones like *Eb*

In [ ]:
play(Fs5)
sleep(0.5)
play(Eb5)

Play louder (parameter amp) or from a different direction (parameter pan)

In [ ]:
play(72,amp=2)
sleep(0.5)
play(74,pan=-1) #left

Different synthesizer sounds

In [ ]:
use_synth(SAW)
play(38)
sleep(0.25)
play(50)
sleep(0.5)
use_synth(PROPHET)
play(57)
sleep(0.25)

ADSR *(Attack, Decay, Sustain and Release)* Envelope

In [ ]:
play (60, attack=0.5, decay=1, sustain_level=0.4, sustain=2, release=0.5) 
sleep(4)

Play some samples

In [ ]:
sample(AMBI_LUNAR_LAND, amp=0.5)

In [ ]:
sample(LOOP_AMEN,pan=-1)
sleep(0.877)
sample(LOOP_AMEN,pan=1)

In [ ]:
sample(LOOP_AMEN,rate=0.5)

In [ ]:
sample(LOOP_AMEN,rate=1.5)

In [ ]:
sample(LOOP_AMEN,rate=-1)#back

In [ ]:
sample(DRUM_CYMBAL_OPEN,attack=0.01,sustain=0.3,release=0.1)

In [ ]:
sample(LOOP_AMEN,start=0.5,finish=0.8,rate=-0.2,attack=0.3,release=1)

Play some random notes

In [ ]:
import random

for i in range(5):
    play(random.randrange(50, 100))
    sleep(0.5)

In [ ]:
for i in range(3):
    play(random.choice([C5,E5,G5]))
    sleep(1)

Sample slicing

In [ ]:
from psonic import *

number_of_pieces = 8

for i in range(16):
    s = random.randrange(0,number_of_pieces)/number_of_pieces #sample starts at 0.0 and finishes at 1.0
    f = s + (1.0/number_of_pieces)
    sample(LOOP_AMEN,beat_stretch=2,start=s,finish=f)
    sleep(2.0/number_of_pieces)

An infinite loop and if

In [ ]:
while True:
  if one_in(2):
    sample(DRUM_HEAVY_KICK)
    sleep(0.5)
  else:
    sample(DRUM_CYMBAL_CLOSED)
    sleep(0.25)

If you want to hear more than one sound at a time, use Threads.

In [ ]:
import random
from psonic import *
from threading import Thread

def bass_sound():
    c = chord(E3, MAJOR7)
    while True:
        use_synth(PROPHET)
        play(random.choice(c), release=0.6)
        sleep(0.5)

def snare_sound():
    while True:
        sample(ELEC_SNARE)
        sleep(1)

bass_thread = Thread(target=bass_sound)
snare_thread = Thread(target=snare_sound)

bass_thread.start()
snare_thread.start()

while True:
    pass

Every function *bass_sound* and *snare_sound* have its own thread. Your can hear them running.

In [ ]:
from psonic import *
from threading import Thread, Condition
from random import choice

def random_riff(condition):
    use_synth(PROPHET)
    sc = scale(E3, MINOR)
    while True:
        s = random.choice([0.125,0.25,0.5])
        with condition:
            condition.wait() #Wait for message
        for i in range(8):
            r = random.choice([0.125, 0.25, 1, 2])
            n = random.choice(sc)
            co = random.randint(30,100)
            play(n, release = r, cutoff = co)
            sleep(s)

def drums(condition):
    while True:
        with condition:
            condition.notifyAll() #Message to threads
        for i in range(16):
            r = random.randrange(1,10)
            sample(DRUM_BASS_HARD, rate=r)
            sleep(0.125)

condition = Condition()
random_riff_thread = Thread(name='consumer1', target=random_riff, args=(condition,))
drums_thread = Thread(name='producer', target=drums, args=(condition,))

random_riff_thread.start()
drums_thread.start()

input("Press Enter to continue...")

To synchronize the thread, so that they play a note at the same time, you can use *Condition*. One function sends a message with *condition.notifyAll* the other waits until the message comes *condition.wait*.

More simple with decorator __@in_thread__

In [ ]:
from psonic import *
from random import choice

tick = Message()

@in_thread
def random_riff():
    use_synth(PROPHET)
    sc = scale(E3, MINOR)
    while True:
        s = random.choice([0.125,0.25,0.5])
        tick.sync()
        for i in range(8):
            r = random.choice([0.125, 0.25, 1, 2])
            n = random.choice(sc)
            co = random.randint(30,100)
            play(n, release = r, cutoff = co)
            sleep(s)
            
@in_thread
def drums():
    while True:
        tick.cue()
        for i in range(16):
            r = random.randrange(1,10)
            sample(DRUM_BASS_HARD, rate=r)
            sleep(0.125)

random_riff()
drums()

input("Press Enter to continue...")

In [ ]:
from psonic import *

tick = Message()

@in_thread
def metronom():
    while True:
        tick.cue()
        sleep(1)
        
@in_thread
def instrument():
    while True:
        tick.sync()
        sample(DRUM_HEAVY_KICK)

metronom()
instrument()

while True:
    pass

Play a list of notes

In [ ]:
from psonic import *

play ([64, 67, 71], amp = 0.3) 
sleep(1)
play ([E4, G4, B4])
sleep(1)

Play chords

In [ ]:
play(chord(E4, MINOR)) 
sleep(1)
play(chord(E4, MAJOR))
sleep(1)
play(chord(E4, MINOR7))
sleep(1)
play(chord(E4, DOM7))
sleep(1)

Play arpeggios

In [ ]:
play_pattern( chord(E4, 'm7')) 
play_pattern_timed( chord(E4, 'm7'), 0.25) 
play_pattern_timed(chord(E4, 'dim'), [0.25, 0.5]) 

Play scales

In [ ]:
play_pattern_timed(scale(C3, MAJOR), 0.125, release = 0.1) 
play_pattern_timed(scale(C3, MAJOR, num_octaves = 2), 0.125, release = 0.1) 
play_pattern_timed(scale(C3, MAJOR_PENTATONIC, num_octaves = 2), 0.125, release = 0.1)

The function *scale* returns a list with all notes of a scale. So you can use list methodes or functions. For example to play arpeggios descending or shuffeld.

In [ ]:
import random

s = scale(C3, MAJOR)
s

In [ ]:
play_pattern_timed(s.reverse(), 0.125, release = 0.1)
play_pattern_timed(random.shuffle(s), 0.125, release = 0.1)

### Live Loop

One of the best in SONIC PI is the _Live Loop_. While a loop is playing music you can change it and hear the change. Let's try it in Python, too.

In [ ]:
from psonic import *
from threading import Thread

def my_loop():
  play(60)
  sleep(1)

def looper():
  while True:
    my_loop()

looper_thread = Thread(name='looper', target=looper)

looper_thread.start()

input("Press Enter to continue...")

Now change the function *my_loop* und you can hear it.

In [ ]:
def my_loop():
  use_synth(TB303)
  play (60, release= 0.3)
  sleep (0.25)

In [ ]:
def my_loop():
  use_synth(TB303)
  play (chord(E3, MINOR), release= 0.3)
  sleep(0.5)

In [ ]:
def my_loop():
    use_synth(TB303)
    sample(DRUM_BASS_HARD, rate = random.uniform(0.5, 2))
    play(random.choice(chord(E3, MINOR)), release= 0.2, cutoff=random.randrange(60, 130))
    sleep(0.25)

To stop the sound you have to end the kernel. In IPython with Kernel --> Restart

Now with two live loops which are synch.

In [ ]:
from psonic import *
from threading import Thread, Condition
from random import choice

def loop_foo():
  play (E4, release = 0.5)
  sleep (0.5)


def loop_bar():
  sample (DRUM_SNARE_SOFT)
  sleep (1)
    

def live_loop_1(condition):
    while True:
        with condition:
            condition.notifyAll() #Message to threads
        loop_foo()
            
def live_loop_2(condition):
    while True:
        with condition:
            condition.wait() #Wait for message
        loop_bar()

condition = Condition()
live_thread_1 = Thread(name='producer', target=live_loop_1, args=(condition,))
live_thread_2 = Thread(name='consumer1', target=live_loop_2, args=(condition,))

live_thread_1.start()
live_thread_2.start()

input("Press Enter to continue...")

In [ ]:
def loop_foo():
  play (A4, release = 0.5)
  sleep (0.5)

In [ ]:
def loop_bar():
  sample (DRUM_HEAVY_KICK)
  sleep (0.125)

If would be nice if we can stop the loop with a simple command. With stop event it works.

In [ ]:
from psonic import *
from threading import Thread, Condition, Event

def loop_foo():
  play (E4, release = 0.5)
  sleep (0.5)


def loop_bar():
  sample (DRUM_SNARE_SOFT)
  sleep (1)
    

def live_loop_1(condition,stop_event):
    while not stop_event.is_set():
        with condition:
            condition.notifyAll() #Message to threads
        loop_foo()
            
def live_loop_2(condition,stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        loop_bar()



condition = Condition()
stop_event = Event()
live_thread_1 = Thread(name='producer', target=live_loop_1, args=(condition,stop_event))
live_thread_2 = Thread(name='consumer1', target=live_loop_2, args=(condition,stop_event))


live_thread_1.start()
live_thread_2.start()

input("Press Enter to continue...")

In [ ]:
stop_event.set()

More complex live loops

In [ ]:
sc = Ring(scale(E3, MINOR_PENTATONIC))

def loop_foo():
  play (next(sc), release= 0.1)
  sleep (0.125)

sc2 = Ring(scale(E3,MINOR_PENTATONIC,num_octaves=2))
           
def loop_bar():
  use_synth(DSAW)
  play (next(sc2), release= 0.25)
  sleep (0.25)

Now a simple structure with four live loops

In [ ]:
import random
from psonic import *
from threading import Thread, Condition, Event

def live_1():
    pass

def live_2():
    pass
    
def live_3():
    pass

def live_4():
    pass

def live_loop_1(condition,stop_event):
    while not stop_event.is_set():
        with condition:
            condition.notifyAll() #Message to threads
        live_1()
            
def live_loop_2(condition,stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        live_2()

def live_loop_3(condition,stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        live_3()

def live_loop_4(condition,stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        live_4()
        
condition = Condition()
stop_event = Event()
live_thread_1 = Thread(name='producer', target=live_loop_1, args=(condition,stop_event))
live_thread_2 = Thread(name='consumer1', target=live_loop_2, args=(condition,stop_event))
live_thread_3 = Thread(name='consumer2', target=live_loop_3, args=(condition,stop_event))
live_thread_4 = Thread(name='consumer3', target=live_loop_3, args=(condition,stop_event))

live_thread_1.start()
live_thread_2.start()
live_thread_3.start()
live_thread_4.start()

input("Press Enter to continue...")

After starting the loops you can change them

In [ ]:
def live_1():
    sample(BD_HAUS,amp=2)
    sleep(0.5)
    pass

In [ ]:
def live_2():
    #sample(AMBI_CHOIR, rate=0.4)
    #sleep(1)
    pass

In [ ]:
def live_3():
    use_synth(TB303)
    play(E2, release=4,cutoff=120,cutoff_attack=1)
    sleep(4)

In [ ]:
def live_4():
    notes = scale(E3, MINOR_PENTATONIC, num_octaves=2)
    for i in range(8):
        play(random.choice(notes),release=0.1,amp=1.5)
        sleep(0.125)

And stop.

In [ ]:
stop_event.set()

## More Examples

In [ ]:
from psonic import *

In [ ]:
#Inspired by Steve Reich Clapping Music

clapping = [1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0]

for i in range(13):
    for j in range(4):
        for k in range(12): 
          if clapping[k] ==1 : sample(DRUM_SNARE_SOFT,pan=-0.5)
          if clapping[(i+k)%12] == 1: sample(DRUM_HEAVY_KICK,pan=0.5)
          sleep (0.25)

## More Informations

### Sonic Pi

..

### OSC

..

### MIDI

..

## Sources

Joe Armstrong: Connecting Erlang to the Sonic Pi http://joearms.github.io/2015/01/05/Connecting-Erlang-to-Sonic-Pi.html

Joe Armstrong: Controlling Sound with OSC Messages http://joearms.github.io/2016/01/29/Controlling-Sound-with-OSC-Messages.html

..